## 라이브러리
- flatten: 2차원 자료를 1차원 자료로 바꿈
- dense : 입력과 출력을 연결해줌
- conv2D : 컨볼루션 필터 적용 함수 

In [1]:
import tensorflow as tf   #

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

## 데이터 준비
-  데이터셋중 mnist 데이터셋 가져옴 

In [5]:
mnist=tf.keras.datasets.mnist
(x_train, y_train),(x_test,y_test)=mnist.load_data()
x_train, x_test=x_train/255.0,x_test/255.0

### 채널 차원추가

In [3]:
x_train=x_train[...,tf.newaxis]    #채널 차원추가 3차원  -> 4차원 
x_test=x_test[...,tf.newaxis]

 ### 데이터셋을 섞고 배치를 만듦

In [7]:
train_ds=tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32)
test_ds=tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)
 # 데이터셋을 섞고 배치를 만듦

### keras의 모델 서브클래스 api를 사용 tf.keras 모델 만듦
    -Model을 상속받음 
    -

In [28]:
class MyModel(Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.conv1=Conv2D(32,3,activation='relu')
        self.flatten=Flatten()
        self.d1=Dense(128,activation='relu')
        self.d2=Dense(10,activation='softmax')
    def call(self,x):
        x=self.conv1(x)
        x=self.flatten(x)
        x=self.d1(x)
        return self.d2(x)
model=MyModel()

### 옵티마이저와 손실함수 선택

In [30]:
loss_object=tf.keras.losses.SparseCategoricalCrossentropy()
optimizer=tf.keras.optimizers.Adam()

### 성능지표 선택, 결과출력

In [31]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

  ### 모델 훈련

In [32]:
@tf.function
def train_step(images,labels):
    with tf.GradientTape() as tape:
        predictions=model(images)
        loss=loss_object(labels,predictions)
    gradients=tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels,predictions)


In [33]:
@tf.function
def test_step(images,labels):
    predictions=model(images)
    t_loss=loss_object(labels,predictions)
    
    test_loss(t_loss)
    test_accuracy(labels,predictions)
    

In [35]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
    print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

에포크: 1, 손실: 0.10833773016929626, 정확도: 96.79389953613281, 테스트 손실: 0.04901259019970894, 테스트 정확도: 98.43999481201172
에포크: 2, 손실: 0.07713216543197632, 정확도: 97.6738052368164, 테스트 손실: 0.048362161964178085, 테스트 정확도: 98.40999603271484
에포크: 3, 손실: 0.059483885765075684, 정확도: 98.19613647460938, 테스트 손실: 0.05300288647413254, 테스트 정확도: 98.32333374023438
에포크: 4, 손실: 0.04869180545210838, 정확도: 98.51197814941406, 테스트 손실: 0.05575716868042946, 테스트 정확도: 98.29999542236328
에포크: 5, 손실: 0.04124029353260994, 정확도: 98.73672485351562, 테스트 손실: 0.056369598954916, 테스트 정확도: 98.33399963378906
